In [3]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("downloads").getOrCreate()

In [10]:
ms_download_facts = spark.read.csv('/home/jovyan/work/Premium vs Freemium/ms_download_facts.csv', header=True, inferSchema=True)

ms_acc_dimension = spark.read.csv('/home/jovyan/work/Premium vs Freemium/ms_acc_dimension.csv', header=True, inferSchema=True)

ms_user_dimension = spark.read.csv('/home/jovyan/work/Premium vs Freemium/ms_user_dimension.csv', header=True, inferSchema=True)

In [11]:
ms_download_facts = ms_download_facts.join(
    ms_user_dimension, ms_download_facts.user_id == ms_user_dimension.user_id, "left" ).join(
        ms_acc_dimension, ms_user_dimension.acc_id == ms_acc_dimension.acc_id)

In [12]:
ms_download_facts = ms_download_facts.groupBy("date", "paying_customer").agg(
    F.sum("downloads").alias("total_downloads")
)

In [13]:
ms_download_facts = ms_download_facts.groupBy("date").pivot("paying_customer").sum("total_downloads")

In [14]:
ms_download_facts = ms_download_facts.withColumnRenamed("no", "non_paying_downloads").withColumnRenamed("yes", "paying_downloads")

In [15]:
ms_download_facts = ms_download_facts.filter(F.col("non_paying_downloads") > F.col("paying_downloads"))

In [16]:
ms_download_facts = ms_download_facts.orderBy(F.col("date").asc())

In [17]:
ms_download_facts = ms_download_facts.select("date", "non_paying_downloads", "paying_downloads")

In [18]:
ms_download_facts.toPandas()

,date,non_paying_downloads,paying_downloads
0,2020-08-16,15,14
1,2020-08-17,45,9
2,2020-08-18,10,7
3,2020-08-21,32,17
